## Text extraction and preparation

The text data source is from PDF documents which are mostly of conference and journal papers.

I use several tools to extract the text from PDF. One package is called **textract**

Some manual intervention (editing, and deletion) are involved as the extraction results are not ideal.

In [1]:
import os
import keras
from keras.preprocessing import text
import textract

Using TensorFlow backend.


Below is one snapshot of the folder location where a list of PDF papers which are in one category are placed into.

In [39]:
path = 'C:/Users/k/Documents/qiqqa/1194C662-D2D4-4D9D-B85E-E461A2F306E5'
pdf = []
for root, dr, fs in os.walk(path):
    for f in fs:
        if '.pdf' not in f:
            continue
        pdf.append(root + '/' + f)

Following, we use combination of **nltk** stopwords and RegexpTokenizer to help remove unwanted characters. Keras text preprocessing is also used to split the texts into word sequences.

In [40]:
# the snippet code from ref[1] -- see next notebook
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer, sent_tokenize

# Load stop-words
stop_words = set(stopwords.words('english'))

# Initialize tokenizer
# It's also possible to try with a stemmer or to mix a stemmer and a lemmatizer
tokenizer = RegexpTokenizer('[\'a-zA-Z]+')

In [41]:
len(pdf)

108

In [42]:
min_num_words = 5
label = 'device characterizations'
docs = []

In [43]:
for doc in pdf:
    try:
        p1 = textract.process(doc)
    except:
        print('exception..')
        pass
    p2 = str(p1)
    p3 = p2.split('\\r\\n')
    print(p3[:5])
    print()
    i = 0
    start = False
    txt = []
    for p in p3:
        # w = [t for t in tokenizer.tokenize(p) if t.lower() not in stop_words]
        w = keras.preprocessing.text.text_to_word_sequence(p)
        if 'abstract' not in w and not start:
            continue
        if 'abstract' in w:
            start = True
            continue
        if 'references' in w:
            break
        w1 = [v for v in w if not v.lower().startswith('x')]
        if len(w1)<min_num_words:
            continue
        for word in w1:
            txt.append(word)
#         print(w1)
        i = i +1
        if i>1000:
            break
    if txt != []:
        docs.append((label,txt))

["b'IEEE TRANSACTIONS ON COMPONENTS, PACKAGING AND MANUFACTURING TECHNOLOGY, VOL. 4, NO. 10, OCTOBER 2014", '', '1589', '', 'Optimal Material Properties of Molding']

["b'www.advopticalmat.de", 'www.MaterialsViews.com', '', 'COMMUNICATION', '']

["b'This article has been accepted for inclusion in a future issue of this journal. Content is final as presented, with the exception of pagination.", 'JOURNAL OF MICROELECTROMECHANICAL SYSTEMS', '', '1', '']

["b'A FLEXIBLE LIQUID CRYSTAL POLYMER MEMS", 'PRESSURE SENSOR ARRAY FOR FISH-LIKE', 'UNDERWATER SENSING', 'A. G. P. Kottapalli, M. Asadnia, J.M. Miao,', 'G. Barbastathis and M. S. Triantafyllou']

["b'International Conference on Smart Materials Structures and Systems", 'January 04-07, 2012, Bangalore, India', '', 'Dynamic Characterization of', 'a Fabricated Dual-axis In-plane Capacitive Accelerometer']

["b'610", '', 'JOURNAL OF MICROELECTROMECHANICAL SYSTEMS, VOL. 23, NO. 3, JUNE 2014', '', 'A Feedback Controlled MEMS Nanopositioner']

[

In [44]:
print(len(docs))

81


Save the text and category as **pkl** file

In [46]:
import _pickle as pickle

In [48]:
pickle.dump(docs, open('device_characterization.pkl', 'wb'))